In [1]:
import itertools
import numpy as np
from pyomo.environ import *
from pyomo.environ import value as pyoval
from pyomo.environ import Binary, NonNegativeReals, UnitInterval
from typing import Tuple, Union
from collections import defaultdict

In [2]:
def active_set_method(A:np.ndarray, b:np.ndarray, theta_nominal:list, theta_deviations:list, design_vectors:np.ndarray | list[np.ndarray]=None, z_bounds:list[Tuple]=None, bigM:float=1e3, integer_cuts:list=None):
    """
    From:
    A(d, z, theta) <= b
    z_lb <= z <= z_ub
    theta_nominal - delta * theta_deviations_lb <= theta <= theta_nominal + delta * theta_deviations_ub
    
    Convert to:
    h(d, z, theta, slack, dual, binary) == b_h
    g(z, theta, slack, dual, binary, delta) <= b_g

    Args:
        A: Coefficient matrix of system equations
        b: Vector of constant values
        theta_nominal: Nominal values of uncertain parameters
        theta_deviations: Allowable changes from the nominal value (lower bound, upper bound)
        design_vectors: Values of design variables
        z_bounds: Bounds on control variables
        bigM: 

    Returns:
        m: Pyomo model
        delta: Flexibility index value
    """
    if not (isinstance(theta_nominal, list) and isinstance(theta_deviations, list) and all(isinstance(t, tuple) and len(t)==2 and all(isinstance(x, (int, float)) for x in t) for t in theta_deviations)):
        raise ValueError('Please provide a list (of length n_t) of tuples (of size 2) and containing only float or integer values!')
    elif len(theta_nominal) != len(theta_deviations):
        raise ValueError('Inconsistent input for nominal theta values and theta bounds')
    
    n_t = len(theta_nominal)
    n_d = design_vectors[0].shape[0] if isinstance(design_vectors, list) else design_vectors.shape[0] if isinstance(design_vectors, np.ndarray) else 0
    n_c = A.shape[1] - n_t - n_d
    if n_c < 0:
        raise ValueError('Inconsistent coefficients input')
    # print(f'num_theta={n_t}, num_design={n_d}, num_control={n_c}')
    # print(f'num_constraints={A.shape[0]}')
    if isinstance(z_bounds, list):        
        if not (len(z_bounds) == n_c and all(isinstance(t, tuple) and len(t)==2 and all(isinstance(x, (int, float)) for x in t) for t in z_bounds)):
            raise ValueError('Please provide a list (of length n_c) of tuples (of size 2) and containing only float or integer values!')
    
    # Reformulate A(d, z, theta) <= b to h(d, z, theta, slack, dual, binary) == b_h
    # h1(d, z, theta, slack) == b               for all system constraints
    # h2(dual) == 0 (from dA/dz = 0)            for all control variables
    # h3(dual, binary) == [1, n_c+1]'           2 constraints
    # d == dE                                   for all design variables - if provided
    # b_h = [b, 0*n_c, 1, n_c+1, dE (if provided)]'
    h1 = np.concatenate([A, np.diag([1]*A.shape[0]), np.zeros((A.shape[0], 2*A.shape[0]))], axis=1)
    h2 = np.concatenate([np.zeros((n_c, A.shape[1]+A.shape[0])), A[:, n_d:n_d+n_c].T, np.zeros((n_c, A.shape[0]))], axis=1)
    h3 = np.concatenate([np.zeros((2, A.shape[1]+A.shape[0])), np.stack([np.ones(A.shape[0]).T, np.zeros(A.shape[0]).T], axis=0), np.stack([np.zeros(A.shape[0]).T, np.ones(A.shape[0]).T], axis=0)], axis=1)
    h = np.concatenate([h1, h2, h3], axis=0)
    b_h = np.concatenate([b, np.zeros(n_c), np.array([1]), np.array([n_c+1])], axis=0)
    
    # Add inequalities for the active set reformulation g(z, theta, slack, dual, binary, delta) <= b_g
    # z <= z_UB                                             Upper bounds for all control variables
    # -z <= -z_LB                                           Lower bounds for all control variables
    # theta - theta_bound*delta <= theta_nominal            Upper bounds for all uncertain parameters
    # -theta - theta_bound*delta <= -theta_nominal          Lower bounds for all uncertain parameters
    # slack + bigM*binary <= bigM                           # bigM reformulation of complementary conditions
    # dual - binary <= 0                                    bigM reformulation of complementary conditions
    g1 = np.concatenate([np.concatenate([np.diag([1]*n_t), np.diag([-1]*n_t)], axis=0), np.zeros((2*n_t, 3*A.shape[0])), np.stack([-np.array([i[1] for i in theta_deviations]), -np.array([i[1] for i in theta_deviations])], axis=0).reshape(-1,1)], axis=1)
    g2 = np.concatenate([np.zeros((A.shape[0], n_t)), np.diag([1]*A.shape[0]), np.diag([0]*A.shape[0]), np.diag([bigM]*A.shape[0]), np.zeros(A.shape[0]).reshape(-1,1)], axis=1)
    g3 = np.concatenate([np.zeros((A.shape[0], n_t)), np.diag([0]*A.shape[0]), np.diag([1]*A.shape[0]), np.diag([-1]*A.shape[0]), np.zeros(A.shape[0]).reshape(-1,1)], axis=1)
    g = np.concatenate([g1, g2, g3], axis=0)
    b_g = np.concatenate([np.array(theta_nominal), -np.array(theta_nominal), np.array([bigM]*A.shape[0]), np.array([0]*A.shape[0])], axis=0)
    g_shift=n_d+n_c
    
    def evaluate_delta_fixed_design(h, b_h, g, b_g, g_shift, z_b:np.ndarray=None, design_vector:np.ndarray=None):
        if isinstance(design_vector, np.ndarray):
            h4 = np.concatenate([np.diag([1]*n_d), np.zeros((n_d, n_c+n_t+3*A.shape[0]))], axis=1)
            h_new, b_h_new = np.concatenate([h, h4], axis=0),np.concatenate([b_h, design_vector], axis=0)
            # b_h_new = np.concatenate([b_h, design_vector], axis=0)
        else:
            h_new, b_h_new= h, b_h
            # b_h_new = b_h
        if isinstance(z_b, list):
            g0 = np.concatenate([np.concatenate([np.diag([1]*n_c), np.diag([-1]*n_c)], axis=0), np.zeros((2*n_c, n_t+3*A.shape[0]+1))], axis=1)
            gz = np.zeros((2*(n_t+A.shape[0]), n_c))
            b0 = np.array([i[1] for i in z_bounds]+[-i[0] for i in z_bounds])
            g_new, b_g_new = np.concatenate([g0, np.concatenate([gz, g], axis=1)], axis=0), np.concatenate([b0, b_g], axis=0)
            g_shift=n_d
        else:
            g_new, b_g_new = g, b_g
        # print(f'Shape of h matrix {h.shape}')
        # print(f'Shape of b_h matrix {b_h.shape}')
        # print(f'Shape of g matrix {g.shape}')
        # print(f'Shape of b_g matrix {b_g.shape}')
                
        binary_indexes = {i for i in range(A.shape[1]+2*A.shape[0]+1, A.shape[1]+3*A.shape[0]+1)}
        def var_domain_rule(instance, i):
            return Binary if i in binary_indexes else NonNegativeReals
        
        m = ConcreteModel()
        m.v = RangeSet(n_d+n_c+n_t+3*A.shape[0]+1)
        m.var = Var(m.v, domain=var_domain_rule)        # order of variables (design, control, theta, slack, dual, binary, delta)
        
        m.hi = RangeSet(h_new.shape[0])
        m.hj = RangeSet(h_new.shape[1])
        m.gi = RangeSet(g_new.shape[0])
        m.gj = RangeSet(g_new.shape[1])

        def h_eq_bh_rule(instance, i):
            return sum(h_new[i-1, j-1]*instance.var[j] for j in instance.hj) == b_h_new[i-1]
        m.h_eq_bh = Constraint(m.hi, rule=h_eq_bh_rule)
        
        def g_leq_bg_rule(instance, i):
            return sum(g_new[i-1, j-1]*instance.var[j+g_shift] for j in instance.gj) <= b_g_new[i-1]
        m.g_leq_bg = Constraint(m.gi, rule=g_leq_bg_rule)
        
        # Objective
        m.objective = Objective(expr=m.var[len(m.v)], sense=minimize)
        solver = SolverFactory('gurobi')
        solver.solve(m)
        
        actives = [j-(A.shape[1]+2*A.shape[0]) for j in binary_indexes if pyoval(m.var[j]) == 1] # Adjust for constraint number
        return m, pyoval(m.var[len(m.v)]), actives
    
    if isinstance(design_vectors, list):
        m, delta, active_set = [defaultdict(dict) for _ in range(3)]
        for i in range(len(design_vectors)):
            m[i], delta[i], active_set[i] = evaluate_delta_fixed_design(h=h, b_h=b_h, g=g, b_g=b_g, g_shift=g_shift, z_b=z_bounds, design_vector=design_vectors[i])
    elif isinstance(design_vectors, np.ndarray):
        m, delta, active_set = evaluate_delta_fixed_design(h=h, b_h=b_h, g=g, b_g=b_g, g_shift=g_shift, z_b=z_bounds, design_vector=design_vectors)
    else:
        m, delta, active_set = evaluate_delta_fixed_design(h=h, b_h=b_h, g=g, b_g=b_g, g_shift=g_shift)
    
    return m, delta, active_set

In [3]:
# EXAMPLE 1 Pistikopoulos and Grossmann (1988): https://www.sciencedirect.com/science/article/pii/0098135488800103
theta_nominal = [2]
theta_deviations = [(2,2)]

A = np.array([
    [1, -3, 1, -1],
    [0, 1, -1, -1/3],
    [-1, 0, 1 ,1]
])

b = np.array([0, -1/3, 1])

d = np.array([3, 1])

In [4]:
model, flex_index, act_set = active_set_method(A=A, b=b, theta_nominal=theta_nominal, theta_deviations=theta_deviations, design_vectors=d)

In [5]:
act_set

[1, 2]

In [6]:
def model_relaxation(m:ConcreteModel):
    relax_m = m
    
    for v in relax_m.component_objects(Var):
        for idx in v:         
            var = v[idx]
            if var.domain is Binary:
                var.domain = UnitInterval
                var.fix(pyoval(var))
    
    relax_m.dual = Suffix(direction=Suffix.IMPORT)
    
    relax_solver = SolverFactory('gurobi')
    relax_results = relax_solver.solve(relax_m)
    
    return relax_m

In [7]:
mrelax = model_relaxation(model)

In [8]:
mrelax.dual.pprint()

dual : Direction=IMPORT, Datatype=FLOAT
    Key         : Value
    g_leq_bg[1] :    0.0
    g_leq_bg[2] :   -0.5
    g_leq_bg[3] : -0.375
    g_leq_bg[4] : -0.375
    g_leq_bg[5] :    0.0
    g_leq_bg[6] :    0.0
    g_leq_bg[7] :    0.0
    g_leq_bg[8] :    0.0
     h_eq_bh[1] :  0.375
     h_eq_bh[2] :  0.375
     h_eq_bh[3] :    0.0
     h_eq_bh[4] :    0.0
     h_eq_bh[5] :    0.0
     h_eq_bh[6] :    0.0
     h_eq_bh[7] : -0.375
     h_eq_bh[8] :   0.75


In [10]:
eq_cons = getattr(mrelax, 'h_eq_bh')

In [12]:
eq_cons.pprint()

h_eq_bh : Size=8, Index=hi, Active=True
    Key : Lower               : Body                                                                                                                                                                          : Upper               : Active
      1 :                 0.0 :                    var[1] - 3.0*var[2] + var[3] - var[4] + var[5] + 0.0*var[6] + 0.0*var[7] + 0.0*var[8] + 0.0*var[9] + 0.0*var[10] + 0.0*var[11] + 0.0*var[12] + 0.0*var[13] :                 0.0 :   True
      2 : -0.3333333333333333 : 0.0*var[1] + var[2] - var[3] - 0.3333333333333333*var[4] + 0.0*var[5] + var[6] + 0.0*var[7] + 0.0*var[8] + 0.0*var[9] + 0.0*var[10] + 0.0*var[11] + 0.0*var[12] + 0.0*var[13] : -0.3333333333333333 :   True
      3 :                 1.0 :                  - var[1] + 0.0*var[2] + var[3] + var[4] + 0.0*var[5] + 0.0*var[6] + var[7] + 0.0*var[8] + 0.0*var[9] + 0.0*var[10] + 0.0*var[11] + 0.0*var[12] + 0.0*var[13] :                 1.0 :   True
      4 :   